In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import stats as st


In [3]:
# TODO: automate data acquisition

In [4]:
# import data
df_cases = pd.read_csv("data/RAW_us_confirmed_cases.csv")
df_deaths_md = pd.read_csv("data/CONVENIENT_us_metadata.csv")
df_masks = pd.read_csv("data/mask-use-by-county.csv")
df_policy = pd.read_csv("data/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv")

print(df_cases.head())

  Province_State   Admin2       UID iso2 iso3  code3    FIPS Country_Region  \
0        Alabama  Autauga  84001001   US  USA    840  1001.0             US   
1        Alabama  Baldwin  84001003   US  USA    840  1003.0             US   
2        Alabama  Barbour  84001005   US  USA    840  1005.0             US   
3        Alabama     Bibb  84001007   US  USA    840  1007.0             US   
4        Alabama   Blount  84001009   US  USA    840  1009.0             US   

         Lat      Long_  ... 10/23/22  10/24/22  10/25/22  10/26/22  10/27/22  \
0  32.539527 -86.644082  ...    18480     18480     18480     18480     18511   
1  30.727750 -87.722071  ...    65895     65895     65895     65895     65973   
2  31.868263 -85.387129  ...     6926      6926      6926      6926      6930   
3  32.996421 -87.125115  ...     7560      7560      7560      7560      7575   
4  33.982109 -86.567906  ...    17286     17286     17286     17286     17320   

   10/28/22  10/29/22  10/30/22  10/31

Here, we narrow the data down, particulary to the county of interst, Pierce County in WA.



In [5]:

p_population = df_deaths_md.query("Province_State == 'Washington' and Admin2 == 'Pierce'")["Population"]
p_masks = df_masks.query("COUNTYFP == 53053")

p_cases_tmp = df_cases.query("Province_State == 'Washington' and Admin2 == 'Pierce'")
id_vars = ["Province_State","Admin2","UID","iso2","iso3","code3","FIPS","Country_Region","Lat","Long_","Combined_Key"]
p_cases = p_cases_tmp.melt(id_vars=id_vars, var_name="date", value_name="cases")

p_policy = df_policy.query("County_Name == 'Pierce County' and State_Tribe_Territory == 'WA'")
p_policy["County"] = "Pierce"
p_policy["masks_mandated"] = p_policy["Face_Masks_Required_in_Public"].apply(lambda x: "yes" if x == "Yes" else "no")



/var/folders/h7/7q35r6p576dg254sr4ltl49r0000gn/T/ipykernel_55962/3192153328.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_policy["County"] = "Pierce"
/var/folders/h7/7q35r6p576dg254sr4ltl49r0000gn/T/ipykernel_55962/3192153328.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_policy["masks_mandated"] = p_policy["Face_Masks_Required_in_Public"].apply(lambda x: "yes" if x == "Yes" else "no")


Here, we perform a rough EDA. 

Observice that the steady increase of cumulative cases, though growth rate varies. 

We also notice 1712 revoked cases, resulting in days with negative case counts. These are zeroed. 

FInally, we observe that, for the data available, once the mask mandate was put into effect, it stayed.

In [6]:
px.bar(p_cases, x="date", y="cases", title="Pierce County Cumulative Cases").show()

p_cases['cases_per_day'] = p_cases['cases'].diff()
p_cases['cases_accumulative'] = p_cases['cases']
px.bar(p_cases, x="date", y="cases_per_day", title="Pierce County Daily Cases", log_y=True).show()

# How many negative cases are there?
case_adjustments = p_cases.query("cases_per_day < 0")
print("Number of cases over counted: ", -case_adjustments['cases_per_day'].sum())

p_cases_no_adjustments = p_cases.query("cases_per_day >= 0")
px.bar(p_cases_no_adjustments, x="date", y="cases_per_day", title="Pierce County Daily Cases (Adjustments removed)", log_y=True).show()

Number of cases over counted:  1712.0


In [7]:
# Now let's try to udnerstand the mask policy
print(p_policy["Face_Masks_Required_in_Public"].unique())
px.line(p_policy, x="date", y="masks_mandated", title="Pierce County Mask Mandate").show()

[nan 'Yes']


Now we try to observe a visual relationship between mask mandate and new infections. I didn't notice a district relationship from the plots above, so I tried to increase the complexity of the model. I am forced to make some big assumptions that are obviously not true:
* Once a person is infected, they cannot be reinfected (I know this is not true, but I don't immediately know how to model it)
* No travel to and from Pierce County.

Infection rate is simply the the number of new cases each day divided by the remaing population that hasn't been infected. 

In [8]:
# Remove infected people from population 
p_cases['county_population'] = p_population.iloc[0]
p_cases.head()

p_cases["uninfected_population"] = p_cases["county_population"] - p_cases["cases_accumulative"]
p_cases["infection_rate"] = (p_cases["cases_per_day"] / p_cases["uninfected_population"]) * 100


In [9]:
px.bar(p_cases, x="date", y="infection_rate", title="Infection Rate", log_y=True).show()

The visualization is not conclusive. While we do see a decline in infection rate after the mandate comes into effect, the best we can do is note the association. The model is extremely simple, and almost certianly misses important, causal variables. 

In [10]:

#print(p_cases.head())
p_cases['date'] = pd.to_datetime(p_cases['date'])
p_cases_clipped = p_cases.query("date <= '2021-8-13'")
print(type(p_cases_clipped['date'].iloc[0]))
p_cases_clipped['mask_mandate'] = p_cases_clipped.apply(lambda x: "yes" if x['date'] > pd.Timestamp("2020-06-26") else "no", axis=1)

px.bar(p_cases_clipped.sort_values(by="date"), 
        x="date", y="infection_rate", color="mask_mandate", 
        title="Infection rate and mask mandate for Pierce County", 
        labels={
            "date": "Date",
            "infection_rate": "Infection Rate (dailyInfections/healthyPopulation)",
            "mask_mandate": "Mask Mandate",
        },
        log_y=True).show()


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


/var/folders/h7/7q35r6p576dg254sr4ltl49r0000gn/T/ipykernel_55962/1201611694.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



EXPANSION
Notes:
* compare cases two weeks after mandate imposed
  * confounding factor: seasonality. Cases go up in winter and go down in summer

* do do R vs D counties: which has more mandates? 
  * who faired worse in covid? 
* 

Is it the masks themselves?
* It doesn't matter. If cases go up or go down, somethign associated with the mandates is change behavior

In [11]:
#df_masks = pd.read_csv("data/mask-use-by-county.csv") # TODO: we may not need this

window_size = 5

# TODO: automate data acquisition for all of these 
df_cases = pd.read_csv("data/RAW_us_confirmed_cases.csv")
df_policy = pd.read_csv("data/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv")
df_election = pd.read_csv("data/dataverse_files/countypres_2000-2020.tab", 
    names=["year", "state", "state_abb", "county", "FIPS", "office", "candidate", "party", "candidate_votes", "total_votes", "mode", "version"], 
    sep="\t")
df_county_pop = pd.read_csv("data/CONVENIENT_us_metadata.csv") # TODO: we may not need this

# We only care about the 2020 election, filter everything else out
df_election = df_election.query("year == 2020")

def election_row(df):
    df.sort_values(by="candidate_votes", ascending=False, inplace=True)
    df["winner_votes"] = df["candidate_votes"].iloc[0]
    df["runnerup_votes"] = df["candidate_votes"].iloc[1]

    df["winner_votes_pct"] = df["winner_votes"] / df["total_votes"]

    return df.iloc[0]
# Select keep only the winning candidate for each county
df_election_winner = df_election.groupby(["FIPS"]).apply(election_row).reset_index(drop=True)

# Convert to standard tabular format
id_vars = ["Province_State","Admin2","UID","iso2","iso3","code3","FIPS","Country_Region","Lat","Long_","Combined_Key"]
p_cases = df_cases.melt(id_vars=id_vars, var_name="date", value_name="cases")
p_cases['date'] = pd.to_datetime(p_cases['date'])

# Cases are cumulative, we need to convert to daily cases per county
p_cases = p_cases.sort_values(by=["FIPS", "date"])
p_cases['cases_cumulative'] = p_cases['cases']
p_cases['cases_per_day'] = p_cases.groupby(["FIPS"])["cases_cumulative"].diff().fillna(0) #p_cases['cases'].diff()
p_cases = p_cases.query("cases_per_day >= 0")
p_cases['cases_per_rolling_rolling'] = p_cases.groupby(["FIPS"])["cases_per_day"].rolling(window_size).mean().reset_index(0, drop=True)

# Set up df county population for later meging
df_county_pop['Province_State'] = df_county_pop['Province_State'].str.upper()
df_county_pop['Admin2'] = df_county_pop['Admin2'].str.upper()

# Prepare policy data for merging
df_policy['County_Name'] = df_policy['County_Name'].str.upper()
df_policy['County_Name'] = df_policy['County_Name'].str.split().str[0] # want just "name" instead of "name county"
df_policy['date'] = pd.to_datetime(df_policy['date'])
df_policy["masks_mandated"] = df_policy["Face_Masks_Required_in_Public"].apply(lambda x: 1 if x == "Yes" else 0)

df_election_winner.head()

,year,state,state_abb,county,FIPS,office,candidate,party,candidate_votes,total_votes,mode,version,winner_votes,runnerup_votes,winner_votes_pct
0,2020,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,19838,27770,20220315,TOTAL,19838,7503,0.714368
1,2020,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,83544,109679,20220315,TOTAL,83544,24578,0.761714
2,2020,ALABAMA,AL,BARBOUR,1005.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,5622,10518,20220315,TOTAL,5622,4816,0.534512
3,2020,ALABAMA,AL,BIBB,1007.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,7525,9595,20220315,TOTAL,7525,1986,0.784263
4,2020,ALABAMA,AL,BLOUNT,1009.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,24711,27588,20220315,TOTAL,24711,2640,0.895716


In [98]:
# combine data sets by county and date
p_cases_elect = p_cases.merge(df_election_winner, left_on=["FIPS"], right_on=["FIPS"])

# now merge in population
p_cases_elect = p_cases_elect.merge(df_county_pop, left_on=["state", "county"], right_on=["Province_State", "Admin2"])

# now merge in mask policy
p_cases_elect = p_cases_elect.merge(
        df_policy, 
        left_on=["state_abb", "county", "date",], 
        right_on=["State_Tribe_Territory", "County_Name", "date",])

p_cases_elect.sort_values(by=["FIPS", "date"], inplace=True)

p_cases_elect["cases_percap"] = p_cases_elect["cases_per_day"] / p_cases_elect["Population"]
p_cases_elect["cases_percap_rolling_rolling"] = p_cases_elect.groupby(["FIPS"])["cases_percap"].rolling(window_size).mean().fillna(0).reset_index(0, drop=True)

# This can generate divide by zeros
p_cases_elect["cases_percap_growth"] = p_cases_elect.groupby(["FIPS"])["cases_percap"].pct_change(14).fillna(0)
p_cases_elect["cases_percap_change_growth"] = p_cases_elect.groupby(["FIPS"])["cases_percap_growth"].pct_change(14).fillna(0)
p_cases_elect["cases_percap_growth_rolling"] = p_cases_elect.groupby(["FIPS"])["cases_percap_change_growth"].rolling(window_size).mean().fillna(0).reset_index(0, drop=True)
p_cases_elect.replace([np.inf, -np.inf], 0, inplace=True)

# keep only the columns we care about
p_cases_elect = p_cases_elect[[
    "date", 
    "cases_per_day", 
    "cases_per_rolling_rolling",
    "cases_percap",
    "cases_percap_rolling_rolling",
    "cases_percap_growth",
    "cases_percap_growth_rolling",
    "masks_mandated",
    "cases_cumulative",
    "FIPS", 
    "county", 
    "state", 
    "state_abb", 
    "party",
    "winner_votes",
    "runnerup_votes",
    "winner_votes_pct",
    "Population",
]]

p_cases_elect = p_cases_elect.sort_values(by=["FIPS", "date"])
p_cases_elect.head()

,date,cases_per_day,cases_per_rolling_rolling,cases_percap,cases_percap_rolling_rolling,cases_percap_growth,cases_percap_growth_rolling,masks_mandated,cases_cumulative,FIPS,county,state,state_abb,party,winner_votes,runnerup_votes,winner_votes_pct,Population
0,2020-04-10,1.0,1.2,0.000018,0.000000,0.0,0.0,0,18,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869
1,2020-04-11,1.0,1.4,0.000018,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869
2,2020-04-12,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869
3,2020-04-13,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869
4,2020-04-14,4.0,1.2,0.000072,0.000021,0.0,0.0,0,23,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869


In [99]:
p_cases_elect.sort_values("cases_percap_growth", ascending=False).head(10)

,date,cases_per_day,cases_per_rolling_rolling,cases_percap,cases_percap_rolling_rolling,cases_percap_growth,cases_percap_growth_rolling,masks_mandated,cases_cumulative,FIPS,county,state,state_abb,party,winner_votes,runnerup_votes,winner_votes_pct,Population
1149521,2021-02-01,2459.0,528.6,0.027715,0.005958,2458.000000,-1.088101,1,8089,48021.0,BASTROP,TEXAS,TX,REPUBLICAN,20516,15474,0.559553,88723
1195583,2021-02-01,1498.0,303.0,0.040874,0.008268,1497.000000,-1.042713,1,4750,48217.0,HILL,TEXAS,TX,REPUBLICAN,11926,2860,0.798741,36649
1150846,2020-10-18,4117.0,834.0,0.011344,0.002298,1371.333333,0.000000,1,6139,48027.0,BELL,TEXAS,TX,REPUBLICAN,67893,57014,0.533017,362924
1157298,2021-02-01,1357.0,285.4,0.028180,0.005927,1356.000000,-0.462985,1,4227,48053.0,BURNET,TEXAS,TX,REPUBLICAN,18767,5639,0.759275,48155
1187802,2021-02-01,1164.0,240.0,0.040305,0.008310,1163.000000,0.000000,1,3771,48185.0,GRIMES,TEXAS,TX,REPUBLICAN,9432,2833,0.759787,28880
700181,2021-03-11,786.0,158.2,0.031760,0.006392,785.000000,0.000000,0,2612,29175.0,RANDOLPH,MISSOURI,MO,REPUBLICAN,8018,2485,0.746277,24748
835306,2020-04-25,762.0,648.2,0.000516,0.000439,761.000000,0.000000,1,31368,36103.0,SUFFOLK,NEW YORK,NY,REPUBLICAN,375821,368000,0.495887,1476601
35387,2021-07-19,750.0,762.4,0.000167,0.000170,749.000000,-144.416051,0,569899,4013.0,MARICOPA,ARIZONA,AZ,DEMOCRAT,978457,889400,0.473109,4485414
225754,2020-11-03,614.0,127.4,0.017183,0.003565,613.000000,-246.252747,0,2382,13299.0,WARE,GEORGIA,GA,REPUBLICAN,5900,2577,0.415727,35734
1172768,2021-01-01,595.0,124.2,0.111611,0.023298,594.000000,-236.866667,1,786,48119.0,DELTA,TEXAS,TX,REPUBLICAN,2162,403,0.834105,5331


In [100]:
# plot political party and cases per capita

# plot political party and mask mandates

# plot change in cases two weeks after mask mandate by political party (only use counties that had a mask mandate)
# Also plot overall change in cases two weeks after mask mandate (all counties)

In [101]:
df_totals_by_party = p_cases_elect.groupby(["date", "party"]).sum().reset_index()
df_totals_by_party["cases_percap"] = df_totals_by_party["cases_per_day"] / df_totals_by_party["Population"]
df_totals_by_party["cases_percap_rolling_rolling"] = df_totals_by_party.groupby(["party"])["cases_percap"].rolling(window_size).mean().reset_index(0, drop=True) 

df_totals_by_party.head()

,date,party,cases_per_day,cases_per_rolling_rolling,cases_percap,cases_percap_rolling_rolling,cases_percap_growth,cases_percap_growth_rolling,masks_mandated,cases_cumulative,FIPS,winner_votes,runnerup_votes,winner_votes_pct,Population
0,2020-04-10,DEMOCRAT,23487.0,22330.8,0.000147,NaN,0.0,0.0,14,354036,13267745.0,44440922,26815421,245.385275,159689772
1,2020-04-10,REPUBLICAN,7828.0,7808.8,0.000063,NaN,0.0,0.0,6,110435,74496374.0,35443955,19484472,1582.408001,123504261
2,2020-04-11,DEMOCRAT,20543.0,22032.6,0.000129,NaN,0.0,0.0,14,374566,13275811.0,44437222,26807995,245.104261,159705031
3,2020-04-11,REPUBLICAN,5692.0,7560.2,0.000046,NaN,0.0,0.0,6,116102,74578500.0,35445194,19454224,1585.195657,123311940
4,2020-04-12,DEMOCRAT,18037.0,21571.8,0.000114,NaN,0.0,0.0,14,392346,13313748.0,44167663,26611762,244.429651,158502494


In [102]:
# plot total democrat population vs republican population
px.bar(df_totals_by_party.query("date == '2020-04-10'"), # this is the first date for which we have some kind of data... which kind? 
    title="Total population by political party",
    x="party", y="Population", color="party",
    width=800, height=600,
).show()

In [103]:
# TODO: adjust plot size, reduce width
fig = make_subplots(rows=2, cols=1, subplot_titles=("Republican", "Democrat"))
fig.add_trace(
    px.bar(df_totals_by_party.query("party == 'REPUBLICAN'"), 
            x="date", y="cases_percap_rolling_rolling", #color="mask_mandate", 
            title="Republican Leaning County daily cases", 
            labels={
                "date": "Date",
            },
            log_y=True).data[0],
    row=1, col=1
)
fig.add_trace(
    px.bar(df_totals_by_party.query("party == 'DEMOCRAT'"), 
            x="date", y="cases_percap_rolling_rolling", #color="mask_mandate", 
            title="Democrat learning County daily cases", 
            labels={
                "date": "Date",
            },
            log_y=True).data[0],
    row=2, col=1
)
fig.show()

In [104]:
# TODO: adjust plot size, reduce width
fig = make_subplots(rows=2, cols=1, subplot_titles=("Republican", "Democrat")) # TODO: maybe try difference between parties? 
fig.add_trace(
    px.bar(df_totals_by_party.query("party == 'REPUBLICAN'"), 
            x="date", y="cases_cumulative",
            labels={
                "date": "Date",
            },
            log_y=True).data[0],
    row=1, col=1
)
fig.add_trace(
    px.bar(df_totals_by_party.query("party == 'DEMOCRAT'"), 
            x="date", y="cases_cumulative",
            labels={
                "date": "Date",
            },
            log_y=True).data[0],
    row=2, col=1
)
fig.show()

In [105]:
# Plot mask mandates per county by party


In [106]:
# Now try to see how mask mandates affect cases

# Compare cases at start of mask mandate to cases two weeks later 
# df_totals_by_party["cases_percap_rolling_rolling"] = df_totals_by_party.groupby(["party"])["cases_percap"].rolling(window_size).mean().reset_index(0, drop=True) 
t_cases_elect = p_cases_elect.copy()

t_cases_elect["mandate_start"] = t_cases_elect["masks_mandated"].diff().fillna(0)
t_cases_elect["ever_mandated"]= t_cases_elect.groupby(["FIPS"])["masks_mandated"].transform("max") # .apply(add_ever_mandated) # .reset_index().query("masks_mandated == 0")["FIPS"]

t_cases_elect.head()

,date,cases_per_day,cases_per_rolling_rolling,cases_percap,cases_percap_rolling_rolling,cases_percap_growth,cases_percap_growth_rolling,masks_mandated,cases_cumulative,FIPS,county,state,state_abb,party,winner_votes,runnerup_votes,winner_votes_pct,Population,mandate_start,ever_mandated
0,2020-04-10,1.0,1.2,0.000018,0.000000,0.0,0.0,0,18,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869,0.0,1
1,2020-04-11,1.0,1.4,0.000018,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869,0.0,1
2,2020-04-12,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869,0.0,1
3,2020-04-13,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869,0.0,1
4,2020-04-14,4.0,1.2,0.000072,0.000021,0.0,0.0,0,23,1001.0,AUTAUGA,ALABAMA,AL,REPUBLICAN,19838,7503,0.714368,55869,0.0,1


In [107]:
# TODO
#   - plot number of people with mask mandate vs number of people without mask mandate
#   - counties that did not implement mask mandates, are the democrat or replublican? 
#   - How long did they last in democrat vs republican counties?
tmp = t_cases_elect.groupby(["date", "ever_mandated"]).sum().reset_index()
tmp["ever_mandated"] = tmp["ever_mandated"].apply(lambda x: "Yes" if x == 1 else "No")
px.bar(tmp.query("date == '2020-04-10'"), # this is the first date for which we have some kind of data... which kind?
    title="Total population that have ever mandated masks",
    x="ever_mandated", y="Population",
    labels={
        "ever_mandated": "Ever subject to mask mandate?",
    },
    width=480, height=300,
).show()
# TODO: clean up x axis (use catagory?)
#

In [108]:
#   - counties that did not implement mask mandates, are the democrat or replublican? 
tmp = t_cases_elect.query("ever_mandated == 0").groupby(["date", "party"]).sum().reset_index()
px.bar(tmp.query("date == '2020-04-10'"), # this is the first date for which we have some kind of data... which kind?
    title="Population in counties that did not mandate a masking policy",
    x="party", y="Population", color="party",
    width=800, height=600,
).show()

In [129]:
# Are mask mandates effective?
# Entire population
t_cases_elect.groupby("FIPS").first().sort_values(by="Population", ascending=True)

t_cases_elect.query("Population > 20000", inplace=True)

In [130]:
cases_masks = t_cases_elect.query("ever_mandated == 1")
mandate_start = cases_masks.query("mandate_start == 1")
def calc_mandate_diffs(delay_in_days=14):
    def calc(df):
        # Get date two weeks from...
        fips = df["FIPS"].iloc[0]
        start_date = mandate_start.query(f"FIPS == {fips}")["date"].iloc[0]
        later_date = start_date + pd.Timedelta(days=delay_in_days)

        max_data = df["date"].max()
        if later_date > max_data:
            later_date = max_data

        row_start = df.query(f"date == '{start_date}'")
        row_later = df.query(f"date == '{later_date}'")

        df["case_diff"] = row_start["cases_per_day"].iloc[0] - row_later["cases_per_day"]#.iloc[0]
        df["case_percap_diff"] = row_start["cases_percap"].iloc[0] - row_later["cases_percap"]#.iloc[0]
        df["case_percap_growth_diff"] = row_start["cases_percap_growth_rolling"].iloc[0] - row_later["cases_percap_growth_rolling"]#.iloc[0]
        df["case_percap_rolling_diff"] = row_start["cases_percap_rolling_rolling"].iloc[0] - row_later["cases_percap_rolling_rolling"]#.iloc[0]
        return df 
    return lambda x: calc(x)

mask_impact = cases_masks.groupby(["state", "county"]).apply(calc_mandate_diffs(delay_in_days=21))
mask_impact.head()

,date,cases_per_day,cases_per_rolling_rolling,cases_percap,cases_percap_rolling_rolling,cases_percap_growth,cases_percap_growth_rolling,masks_mandated,cases_cumulative,FIPS,...,winner_votes,runnerup_votes,winner_votes_pct,Population,mandate_start,ever_mandated,case_diff,case_percap_diff,case_percap_growth_diff,case_percap_rolling_diff
0,2020-04-10,1.0,1.2,0.000018,0.000000,0.0,0.0,0,18,1001.0,...,19838,7503,0.714368,55869,0.0,1,NaN,NaN,NaN,NaN
1,2020-04-11,1.0,1.4,0.000018,0.000000,0.0,0.0,0,19,1001.0,...,19838,7503,0.714368,55869,0.0,1,NaN,NaN,NaN,NaN
2,2020-04-12,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,...,19838,7503,0.714368,55869,0.0,1,NaN,NaN,NaN,NaN
3,2020-04-13,0.0,1.4,0.000000,0.000000,0.0,0.0,0,19,1001.0,...,19838,7503,0.714368,55869,0.0,1,NaN,NaN,NaN,NaN
4,2020-04-14,4.0,1.2,0.000072,0.000021,0.0,0.0,0,23,1001.0,...,19838,7503,0.714368,55869,0.0,1,NaN,NaN,NaN,NaN


In [131]:
# TODO: does masking protect against spikes? 
# TODO: plot cases per capita for counties that have ever mandated masks vs counties that have never mandated masks

def print_stats(df_in, message=""):
    print(message)
    df = df_in.groupby("FIPS").first().reset_index()
    print("Mean case diff: ", df["case_diff"].mean())
    print("Mean case per capita diff: ", df["case_percap_diff"].mean())
    print("Mean case per capita growth diff: ", df["case_percap_growth_diff"].mean())
    print("Mean case per capita rolling diff: ", df["case_percap_rolling_diff"].mean())

def create_map(vt, delay, part, mean, ttest):
    return {
        "voter_threshold": vt,
        "delay": delay,
        "party": part,
        "mean": mean,
        "t-statistic": ttest.statistic,
        "pvalue": ttest.pvalue,
    }
    
def analyze(cases_with_mandates, column_diff, voter_thresholds=[0.30], delay_in_dayss=[14]):
    my_df = []
    for voter_threshold in voter_thresholds:
        for delay_in_days in delay_in_dayss:
            mask_impact = cases_with_mandates.groupby(["state", "county"]).apply(calc_mandate_diffs(delay_in_days=delay_in_days))
            mask_impact = mask_impact.groupby("FIPS").first().reset_index()

            mask_impact.dropna(inplace=True) # TODO: verify this isn't dropping anything we don't need

            all = mask_impact[column_diff]
            dem = mask_impact.query("party == 'DEMOCRAT'")[column_diff]
            rep = mask_impact.query("party == 'REPUBLICAN'")[column_diff]

            my_df.append(create_map(voter_threshold, delay_in_days, "ALL", 
                all.mean(), st.ttest_1samp(all, 0)))

            my_df.append(create_map(voter_threshold, delay_in_days, "DEMOCRAT",
                dem.mean(), st.ttest_1samp(dem, 0)))

            my_df.append(create_map(voter_threshold, delay_in_days, "REPUBLICAN",
                rep.mean(), st.ttest_1samp(dem, 0)))

            my_df.append(create_map(voter_threshold, delay_in_days, "DEMOCRAT vs REPUBLICAN",
                np.NaN, st.ttest_ind(a=dem, b=rep)))

    return pd.DataFrame(my_df) 

df_results_growth = analyze(
        cases_masks, 
        "case_percap_growth_diff",
        voter_thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
        delay_in_dayss=[14, 21, 28])

df_results_cases_window = analyze(
        cases_masks, 
        "case_percap_rolling_diff",
        voter_thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
        delay_in_dayss=[14, 21, 28])


In [133]:
df_results_growth.head()
df_results_growth.sort_values(by='pvalue', ascending=True).head(20)

,voter_threshold,delay,party,mean,t-statistic,pvalue
20,0.4,28,ALL,0.249066,1.468121,0.142519
56,0.7,28,ALL,0.249066,1.468121,0.142519
68,0.8,28,ALL,0.249066,1.468121,0.142519
32,0.5,28,ALL,0.249066,1.468121,0.142519
80,0.9,28,ALL,0.249066,1.468121,0.142519
44,0.6,28,ALL,0.249066,1.468121,0.142519
8,0.3,28,ALL,0.249066,1.468121,0.142519
72,0.9,14,ALL,0.219374,1.450744,0.147298
36,0.6,14,ALL,0.219374,1.450744,0.147298
12,0.4,14,ALL,0.219374,1.450744,0.147298


In [134]:
df_results_cases_window.query('pvalue < 0.05').groupby("delay").first().reset_index().sort_values(by=['pvalue', 'voter_threshold'], ascending=True)

,delay,voter_threshold,party,mean,t-statistic,pvalue
0,28,0.3,ALL,0.000028,3.506219,0.000483
